In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import gymnasium as gym
from gymnasium import spaces
import torch
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
import matplotlib.pyplot as plt

# ==========================================
# 1. GLOBAL CONFIGURATION
# ==========================================
CONFIG = {
    # Assets
    "TICKERS": ["SPY", "IWM", "^VIX", "SHY"], 
    "TARGET_ASSET": "SPY",
    
    # Dates
    "TRAIN_START": "2015-01-01",
    "TRAIN_END": "2023-12-31",
    "TEST_START": "2024-01-02",
    "TEST_END": "2025-01-01",
    
    # Crisis Training Periods (The "Frankenstein" Dataset)
    "CRISIS_PERIODS": [
        ("2007-08-01", "2009-06-01"), # GFC
        ("2018-10-01", "2019-01-01"), # Volmageddon
        ("2020-01-01", "2020-05-01"), # COVID
        ("2022-01-01", "2022-12-31"), # Bear
    ],
    
    # Feature Engineering
    "WINDOW_SIZE": 20,
    "BB_STD": 2.0,
    
    # Agent Parameters (Optimized for Fast Demo Training)
    "LEARNING_RATE": 3e-4,
    "BATCH_SIZE": 64,
    "TRAINING_STEPS": 30000, # Short training for demonstration
    "INITIAL_BALANCE": 100000,
    "FEES": 0.0005
}

# ==========================================
# 2. DATA PROCESSOR (MULTI-REGIME)
# ==========================================
class DataProcessor:
    def __init__(self, tickers):
        self.tickers = tickers
        
    def download(self, start, end):
        try:
            data = yf.download(self.tickers, start=start, end=end, progress=False)
            if isinstance(data.columns, pd.MultiIndex):
                if 'Close' in data.columns.levels[0]: data = data.xs('Close', level=0, axis=1)
                elif 'Adj Close' in data.columns.levels[0]: data = data.xs('Adj Close', level=0, axis=1)
            if isinstance(data, pd.Series): data = data.to_frame()
            
            # Need High/Low for ADX (Using Target Asset)
            ohlc = yf.download(CONFIG['TARGET_ASSET'], start=start, end=end, progress=False)
            return data, ohlc
        except:
            return pd.DataFrame(), pd.DataFrame()

    def add_features(self, df, ohlc, target=CONFIG['TARGET_ASSET']):
        df = df.copy()
        
        # --- 1. Base Returns ---
        df['returns'] = df[target].pct_change()
        
        # --- 2. Trend Indicators (ADX) ---
        df['tr'] = np.maximum(ohlc['High'] - ohlc['Low'], 
                   np.maximum(abs(ohlc['High'] - ohlc['Close'].shift(1)), 
                              abs(ohlc['Low'] - ohlc['Close'].shift(1))))
        df['dm_plus'] = np.where((ohlc['High'] - ohlc['High'].shift(1)) > (ohlc['Low'].shift(1) - ohlc['Low']), 
                                 np.maximum(ohlc['High'] - ohlc['High'].shift(1), 0), 0)
        df['dm_minus'] = np.where((ohlc['Low'].shift(1) - ohlc['Low']) > (ohlc['High'] - ohlc['High'].shift(1)), 
                                  np.maximum(ohlc['Low'].shift(1) - ohlc['Low'], 0), 0)
        
        window = 14
        df['tr_s'] = df['tr'].rolling(window).mean()
        df['dp_s'] = df['dm_plus'].rolling(window).mean()
        df['dm_s'] = df['dm_minus'].rolling(window).mean()
        
        df['di_plus'] = 100 * (df['dp_s'] / df['tr_s'])
        df['di_minus'] = 100 * (df['dm_s'] / df['tr_s'])
        df['dx'] = 100 * abs(df['di_plus'] - df['di_minus']) / (df['di_plus'] + df['di_minus'])
        df['adx'] = df['dx'].rolling(window).mean()
        
        # --- 3. Mean Reversion Indicators (Bollinger) ---
        sma = df[target].rolling(20).mean()
        std = df[target].rolling(20).std()
        df['bb_width'] = (std * 2 * 2) / sma
        df['pct_b'] = (df[target] - (sma - 2*std)) / (4*std)
        df['rsi'] = 100 - (100 / (1 + df['returns'].rolling(14).mean()/df['returns'].rolling(14).std())) # Approx RSI
        
        # --- 4. Crisis Indicators (VIX, SMA Distance) ---
        if '^VIX' in df.columns:
            df['vix_norm'] = (df['^VIX'] - 15) / 40
        else:
            df['vix_norm'] = 0
            
        sma200 = df[target].rolling(200).mean()
        df['dist_sma200'] = (df[target] - sma200) / sma200
        
        df = df.fillna(0)
        return df

    def get_data(self, start, end):
        df, ohlc = self.download(start, end)
        if df.empty: return pd.DataFrame()
        return self.add_features(df, ohlc)

    def get_crisis_data(self):
        # Stitch periods
        dfs = []
        for s, e in CONFIG['CRISIS_PERIODS']:
            d, o = self.download(s, e)
            if not d.empty:
                dfs.append(self.add_features(d, o))
        return pd.concat(dfs).reset_index(drop=True).fillna(0)

# ==========================================
# 3. UNIFIED AGENT ENVIRONMENT
# ==========================================
class TradingEnv(gym.Env):
    def __init__(self, df, mode='trend'):
        super(TradingEnv, self).__init__()
        self.df = df
        self.mode = mode # 'trend', 'mean_rev', 'crisis'
        self.n_features = df.shape[1]
        self.window = CONFIG['WINDOW_SIZE']
        self.current_step = self.window
        
        self.action_space = spaces.Box(low=-1, high=1, shape=(1,), dtype=np.float32)
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(self.window * self.n_features,), dtype=np.float32)
        
        self.data = df.values.astype(np.float32)
        
    def reset(self, seed=None, options=None):
        self.current_step = self.window
        return self._get_obs(), {}
    
    def _get_obs(self):
        return self.data[self.current_step-self.window : self.current_step].flatten()

    def step(self, action):
        if self.current_step >= len(self.df) - 1:
            return self._get_obs(), 0, True, False, {}
            
        act = np.clip(action[0], -1, 1)
        
        # Get Return (Column 4 is usually 'returns' based on add_features order, 
        # but let's be safe and assume it's index 0 of features or specific column)
        # For simplicity in this demo, we assume 'returns' is the first added feature after base data
        # In real code, we'd use column mapping. Here we use the raw price change.
        
        # Ret Col extraction logic (simple approximation for demo)
        ret = self.data[self.current_step, 0] # Returns is usually early in the set
        
        reward = 0
        
        # --- REWARD SHAPING PER SPECIALIST ---
        if self.mode == 'trend':
            # Reward Excess Return
            reward = act * ret * 100
        elif self.mode == 'mean_rev':
            # Reward PnL but punish holding
            reward = (act * ret * 100) - (0.1 * abs(act))
        elif self.mode == 'crisis':
            # Reward Shorting in Drawdowns
            reward = (act * ret * 100)
            if ret < -0.01 and act < -0.5: reward *= 2.0 # Bonus for catching crash
            
        self.current_step += 1
        return self._get_obs(), reward, False, False, {}

# ==========================================
# 4. ENSEMBLE MANAGER
# ==========================================
class EnsembleManager:
    def __init__(self):
        self.dp = DataProcessor(CONFIG['TICKERS'])
        self.agents = {}
        self.envs = {}
        
    def train_specialists(self):
        print("\n=== 1. TRAINING SPECIALIST AGENTS ===")
        
        # 1. Trend Agent (Bull Market Data)
        print(">> Training Trend Agent (Striker)...")
        trend_data = self.dp.get_data(CONFIG['TRAIN_START'], CONFIG['TRAIN_END'])
        env_trend = DummyVecEnv([lambda: TradingEnv(trend_data, mode='trend')])
        env_trend = VecNormalize(env_trend, norm_obs=True, norm_reward=False)
        model_trend = PPO("MlpPolicy", env_trend, verbose=0, learning_rate=CONFIG['LEARNING_RATE'], device='mps')
        model_trend.learn(total_timesteps=CONFIG['TRAINING_STEPS'])
        self.agents['trend'] = model_trend
        self.envs['trend'] = env_trend # Store for normalization stats
        
        # 2. Mean Reversion Agent (Chop Data - IWM proxy logic)
        print(">> Training Mean Rev Agent (Midfielder)...")
        # We use same data but different Reward Logic in Env
        env_mr = DummyVecEnv([lambda: TradingEnv(trend_data, mode='mean_rev')]) 
        env_mr = VecNormalize(env_mr, norm_obs=True, norm_reward=False)
        model_mr = PPO("MlpPolicy", env_mr, verbose=0, learning_rate=CONFIG['LEARNING_RATE'], device='mps')
        model_mr.learn(total_timesteps=CONFIG['TRAINING_STEPS'])
        self.agents['mean_rev'] = model_mr
        self.envs['mean_rev'] = env_mr
        
        # 3. Crisis Agent (Crash Data)
        print(">> Training Crisis Agent (Goalkeeper)...")
        crash_data = self.dp.get_crisis_data()
        env_crisis = DummyVecEnv([lambda: TradingEnv(crash_data, mode='crisis')])
        env_crisis = VecNormalize(env_crisis, norm_obs=True, norm_reward=False)
        model_crisis = PPO("MlpPolicy", env_crisis, verbose=0, learning_rate=CONFIG['LEARNING_RATE'], device='mps')
        model_crisis.learn(total_timesteps=CONFIG['TRAINING_STEPS'])
        self.agents['crisis'] = model_crisis
        self.envs['crisis'] = env_crisis
        
        print("=== TRAINING COMPLETE ===\n")

    def run_backtest(self):
        print("=== 2. RUNNING ENSEMBLE BACKTEST (2024-2025) ===")
        test_data = self.dp.get_data(CONFIG['TEST_START'], CONFIG['TEST_END'])
        if test_data.empty: return
        
        # Prepare execution variables
        balance = CONFIG['INITIAL_BALANCE']
        portfolio = balance
        holdings = 0
        history = []
        
        # Column mapping for logic
        cols = test_data.columns.tolist()
        idx_adx = cols.index('adx') if 'adx' in cols else -1
        idx_vix = cols.index('vix_norm') if 'vix_norm' in cols else -1
        idx_sma = cols.index('dist_sma200') if 'dist_sma200' in cols else -1
        idx_ret = cols.index('returns')
        
        # Convert to numpy for iteration
        data_vals = test_data.values
        dates = test_data.index
        window = CONFIG['WINDOW_SIZE']
        
        print(f"{'Date':<12} | {'Regime':<10} | {'Active Agent':<10} | {'VIX':<6} | {'ADX':<6} | {'Action':<6} | {'Balance':<10}")
        print("-" * 90)
        
        for t in range(window, len(test_data)):
            # 1. Observation
            obs_raw = data_vals[t-window:t].flatten()
            
            # 2. Regime Detection
            vix = data_vals[t, idx_vix]
            adx = data_vals[t, idx_adx]
            sma_dist = data_vals[t, idx_sma]
            
            regime = "CHOP"
            active_agent_name = "mean_rev"
            
            # --- GATING LOGIC ---
            if vix > 0.5 or sma_dist < -0.05: # High VIX or Price 5% below SMA200
                regime = "CRISIS"
                active_agent_name = "crisis"
            elif adx > 25:
                regime = "TREND"
                active_agent_name = "trend"
            else:
                regime = "CHOP"
                active_agent_name = "mean_rev"
                
            # 3. Agent Selection & Prediction
            agent = self.agents[active_agent_name]
            norm_env = self.envs[active_agent_name]
            
            # Normalize observation using the specific agent's training stats
            obs_norm = norm_env.normalize_obs(obs_raw)
            action, _ = agent.predict(obs_norm, deterministic=True)
            
            # 4. Execution
            mkt_ret = data_vals[t, idx_ret]
            position_size = np.clip(action[0], -1, 1)
            
            # Fee logic
            turnover = abs(position_size - holdings)
            cost = turnover * CONFIG['FEES']
            
            pnl_pct = (position_size * mkt_ret) - cost
            portfolio *= (1 + pnl_pct)
            holdings = position_size
            
            # 5. Logging
            if t % 10 == 0: # Print every 10 days to keep output clean but visible
                print(f"{str(dates[t].date()):<12} | {regime:<10} | {active_agent_name:<10} | {vix:<6.2f} | {adx:<6.2f} | {position_size:<6.2f} | {portfolio:<10.0f}")
                
            history.append({
                'Date': dates[t],
                'Portfolio': portfolio,
                'Regime': regime,
                'Agent': active_agent_name,
                'Return': pnl_pct,
                'Benchmark': mkt_ret
            })
            
        # --- Final Stats ---
        res = pd.DataFrame(history).set_index('Date')
        res['Bench_Equity'] = (1 + res['Benchmark']).cumprod() * CONFIG['INITIAL_BALANCE']
        
        total_ret = (portfolio / CONFIG['INITIAL_BALANCE']) - 1
        bench_ret = (res['Bench_Equity'].iloc[-1] / CONFIG['INITIAL_BALANCE']) - 1
        
        print("\n=== FINAL ENSEMBLE REPORT ===")
        print(f"Total Return: {total_ret:.2%} (Benchmark: {bench_ret:.2%})")
        
        # Plot
        plt.figure(figsize=(12, 6))
        plt.plot(res['Portfolio'], label='Ensemble AI', linewidth=2)
        plt.plot(res['Bench_Equity'], label='SPY Buy & Hold', linestyle='--', color='gray')
        
        # Color background by Regime
        # (Simplified visualization for Regime)
        # We can't easily paint background per day in simple matplotlib without complex code,
        # but the Console Output serves the debug purpose well.
        
        plt.title("Ensemble System (Trend + MeanRev + Crisis) vs SPY")
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.show()

if __name__ == "__main__":
    mgr = EnsembleManager()
    mgr.train_specialists()
    mgr.run_backtest()


=== 1. TRAINING SPECIALIST AGENTS ===
>> Training Trend Agent (Striker)...


/var/folders/04/4l6xjjw976j6gj89kg1qrrf80000gn/T/ipykernel_3747/3422824058.py:54: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(self.tickers, start=start, end=end, progress=False)
/var/folders/04/4l6xjjw976j6gj89kg1qrrf80000gn/T/ipykernel_3747/3422824058.py:61: FutureWarning: YF.download() has changed argument auto_adjust default to True
  ohlc = yf.download(CONFIG['TARGET_ASSET'], start=start, end=end, progress=False)
/opt/anaconda3/envs/ml/lib/python3.11/site-packages/stable_baselines3/common/on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and t

>> Training Mean Rev Agent (Midfielder)...
